In [2]:
import os
import getpass
import json

from langchain_core.messages import ToolMessage
from langchain_tavily import TavilySearch
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict

In [3]:
# 🔐 Securely set API keys
os.environ["TAVILY_API_KEY"] = getpass.getpass("🔑 Tavily API Key: ")
os.environ["OPENAI_API_KEY"] = getpass.getpass("🔑 OpenAI API Key: ")

🔑 Tavily API Key:  ········
🔑 OpenAI API Key:  ········


In [4]:
# Tools & Model
tool = TavilySearch(max_results=2)
llm = ChatOpenAI(model="gpt-3.5-turbo").bind_tools([tool])

In [10]:
# 🤖 Chat node (define this BEFORE using in graph)
def chat_node(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

In [11]:
# 🧭 Routing logic
def route(state: State):
    return "tools" if getattr(state["messages"][-1], "tool_calls", None) else "end"


In [12]:
# ⚙️ Tool execution node
class ToolNode:
    def __init__(self, tools): self.tools = {t.name: t for t in tools}
    def __call__(self, state):
        return {
            "messages": [
                ToolMessage(
                    content=json.dumps(self.tools[call["name"]].invoke(call["args"])),
                    name=call["name"],
                    tool_call_id=call["id"]
                ) for call in getattr(state["messages"][-1], "tool_calls", [])
            ]
        }

In [13]:
#Router function
def router(state: State) -> str:
    last_msg = state["messages"][-1]
    return "tools" if getattr(last_msg, "tool_calls", None) else "end"

In [14]:
# Graph
graph = StateGraph(State)
graph.add_node("chatbot", chat_node)
graph.add_node("tools", ToolNode([tool]))
graph.add_node("end", lambda _: {})
graph.add_conditional_edges("chatbot", route, {"tools": "tools", "end": "end"})
graph.add_edge("tools", "chatbot")
graph.set_entry_point("chatbot")
graph.set_finish_point("end")
app = graph.compile()
app.get_graph().print_ascii()

      +-----------+       
      | __start__ |       
      +-----------+       
            *             
            *             
            *             
      +---------+         
      | chatbot |         
      +---------+         
        **     ..         
       *         .        
      *           .       
+-------+       +-----+   
| tools |       | end |   
+-------+       +-----+   
                    *     
                    *     
                    *     
              +---------+ 
              | __end__ | 
              +---------+ 


In [15]:
# Chat loop
def chat():
    while True:
        try:
            q = input("You: ")
            if q.lower() in {"q", "quit", "exit"}: print("👋 Goodbye!"); break
            resp = "".join(
                m.content for e in app.stream({"messages": [{"role": "user", "content": q}]})
                for v in e.values() for m in v["messages"]
            )
            print("Assistant:", resp)
        except Exception as e:
            print("❌", e); breaksti

if __name__ == "__main__": chat()

You:  Give some ideas about unique life style


Assistant: {"query": "unique lifestyle ideas", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.quora.com/What-are-some-unique-and-interesting-ways-to-live-life", "title": "What are some unique and interesting ways to live life? - Quora", "content": "Keep life as simple as possible. Be a share holder for bringing happiness in others life. Never stop chasing your dream till you fulfill it.", "score": 0.54303354, "raw_content": null}, {"url": "https://margarethamontagu.com/15-unique-and-unconventional-ideas-to-help-you-through-lifes-challenges/", "title": "15 Unique and Unconventional Ideas to Help You Through Life's ...", "content": "15 Powerful Suggestions to Make Coping with Life Transitions Easier \u00b7 1. Create a \u201cJust-Because\u201d Gift Basket/Box \u00b7 2. Leave Encouraging Notes in", "score": 0.49641174, "raw_content": null}], "response_time": 1.69}I found some unique lifestyle ideas for you:

1. **Keep Life Simple:** Try to simpl

You:  q


👋 Goodbye!
